In [ ]:
#TASK 1 : TEXT GENERATION WITH GPT-2
!pip install transformers datasets accelerate


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
texts = [
    "Once upon a time, in a magical forest, there lived a wise old owl.",
    "The stars shone brightly over the enchanted mountains.",
    "A dragon flew across the sky, leaving a trail of sparkling fire.",
    "In the ancient library, books whispered secrets of the past.",
    "The mysterious river glowed under the moonlight."
]

dataset = Dataset.from_dict({"text": texts})

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=50)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,       # increase for better results
    per_device_train_batch_size=2,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    prediction_loss_only=True,
    learning_rate=5e-5
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
prompt = "In the mystical forest"
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In the mystical forest, a good time to be the one.

"But, if you're not happy, then I don't know what to do."
"I'm not happy, but I do know I'd like to do this, but I'd like to do this.
"But, if you're not happy, then I don't know what to do I do?
"And, if you're not happy, then I don't know what to do."



In [ ]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os


os.environ["WANDB_DISABLED"] = "true"


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


texts = [
    "Once upon a time, in a magical forest, there lived a wise old owl.",
    "The stars shone brightly over the enchanted mountains.",
    "A dragon flew across the sky, leaving a trail of sparkling fire.",
    "In the ancient library, books whispered secrets of the past.",
    "The mysterious river glowed under the moonlight."
]

dataset = Dataset.from_dict({"text": texts})


def tokenize_and_add_labels(batch):
    tokens = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=50)
    tokens["labels"] = tokens["input_ids"].copy()  # GPT-2 predicts the same tokens
    return tokens

tokenized_dataset = dataset.map(tokenize_and_add_labels, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    prediction_loss_only=True,
    learning_rate=5e-5
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)


trainer.train()


model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

prompt = "In the mystical forest"
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_length=150,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=0.8,
    num_return_sequences=3
)

for i, output in enumerate(outputs):
    print(f"\n--- Generated Text {i+1} ---\n")
    print(tokenizer.decode(output, skip_special_tokens=True))


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.904300
20,0.543900
30,0.321200


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Generated Text 1 ---

In the mystical forest, there lived a wise old owl, who spoke the mysteries of the past.

--- Generated Text 2 ---

In the mystical forest, there lurked a dark secret.

--- Generated Text 3 ---

In the mystical forest, there lived a lost fairy.
